## Imports

In [1]:
# Read stocks
import yfinance as yf

# For DataFrame
import pandas as pd
import numpy as np

# For parsing finviz
import requests
from bs4 import BeautifulSoup

# to calculate std and mean
import statistics

## Ratio Categories

In [2]:
# Scores for Catgeory 1 ratios - lower the better
CAT1_RATIOS = ['D2E', 'PEG', 'PE fwd', 'PB', 'Beta']

# Scores for Catgeory 2 ratios - higher the better
CAT2_RATIOS = ['ROCE', 'ROE', 'FCFY', 'CR', 'QR', 'Asset TR', 'EPS fwd']

## Stock Symbols

In [3]:
def get_symbols():
    req = requests.get('https://finviz.com/screener.ashx',
        params={
            'v': '111',
            'f': 'cap_midover,ind_utilitiesregulatedgas',
            'o': 'company',
        },
        headers={
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        })
    # Creating BeautifulSoup object
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # Table we are interested
    table = soup.find('table', class_='styled-table-new is-rounded is-tabular-nums w-full screener_table')
    # Array to collect symbols
    symbols = []
    for i, row in enumerate(table.find_all('tr')):
        # Skip the header row
        if i != 0:
            # Loop through the row
            for j, td in enumerate(row.find_all('td')):
                # Symbol is in the second column
                if j == 1:
                    symbols.append(td.text.strip())
                    break
    return symbols

## Get Stock symbols

In [4]:
symbols = get_symbols()
# symbols = ['ATO', 'GOOG', 'CPK']

## Utility method to convert a number to human readable format
#### https://stackoverflow.com/questions/579310/formatting-long-numbers-as-strings

In [5]:
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0.'), ['', 'K', 'M', 'B', 'T'][magnitude])

## Ratios using stock info

In [6]:
def populate_with_info(data, stock_info):
    # print(stock_info)
    data['Symbol'].append(stock_info['symbol'])
    data['Name'].append(stock_info['longName'])
    # Convert numbers to a human readable format
    data['Market Cap'].append(human_format(stock_info['marketCap']))
    data['Price'].append(stock_info['currentPrice'])

    # Could be that some indicators are not available; use NaN if this is the case

    # Valuation ratios
    
    if 'priceToBook' in stock_info:
        data['PB'].append(stock_info['priceToBook'])
    else:
        data['PB'].append(np.nan)
    
    if 'forwardEps' in stock_info:
        data['EPS fwd'].append(stock_info['forwardEps'])
    else:
        data['EPS fwd'].append(np.nan)
        
    if 'forwardPE' in stock_info:
        data['PE fwd'].append(stock_info['forwardPE'])
    else:
        data['PE fwd'].append(np.nan)
        
    if 'pegRatio' in stock_info:
        data['PEG'].append(stock_info['pegRatio'])
    else:
        data['PEG'].append(np.nan)
        
    # Solvency financial ratios

    if 'debtToEquity' in stock_info:
        data['D2E'].append(stock_info['debtToEquity'])
    else:
        data['D2E'].append(np.nan)

    # Profitability Ratios
    
    if 'returnOnEquity' in stock_info:
        data['ROE'].append(stock_info['returnOnEquity'])
    else:
        data['ROE'].append(np.nan)
    
    if ('freeCashflow' in stock_info) and ('marketCap' in stock_info):
        fcfy = (stock_info['freeCashflow']/stock_info['marketCap']) * 100
        data['FCFY'].append(round(fcfy, 2))
    else:
        data['FCFY'].append(np.nan)

    # Liquidity ratios

    if 'currentRatio' in stock_info:
        data['CR'].append(stock_info['currentRatio'])
    else:
        data['CR'].append(np.nan)

    if 'quickRatio' in stock_info:
        data['QR'].append(stock_info['quickRatio'])
    else:
        data['CR'].append(np.nan)

    # Other info (non ratios)
    
    if 'dividendYield' in stock_info:
        data['DY'].append(stock_info['dividendYield']*100)
    else:
        data['DY'].append(0.0)

    if 'beta' in stock_info:
        data['Beta'].append(stock_info['beta'])
    else:
        data['Beta'].append(np.nan)

    if 'fiftyTwoWeekLow' in stock_info:
        data['52w Low'].append(stock_info['fiftyTwoWeekLow'])
    else:
        data['52w Low'].append(np.nan)
        
    if 'fiftyTwoWeekHigh' in stock_info:    
        data['52w High'].append(stock_info['fiftyTwoWeekHigh'])
    else:
        data['52w High'].append(np.nan)

## Utility Ratio methods

In [7]:
def roce(ticker):
    income_stm = ticker.income_stmt
    ebit = income_stm.loc['EBIT'].iloc[0]
    bs = ticker.balance_sheet
    return ebit/(bs.loc['Total Assets'].iloc[0]-bs.loc['Current Liabilities'].iloc[0])

def asset_turnover_ratio(ticker):
    df_bs = ticker.balance_sheet
    y0, y1 = df_bs.loc['Total Assets'].iloc[0], df_bs.loc['Total Assets'].iloc[1]
    avg_asset = (y0 + y1)/2
    
    tot_rvn_y0 = ticker.income_stmt.loc['Total Revenue'].iloc[0]/avg_asset

    return tot_rvn_y0

def investory_turnover_ratio(ticker):
    df_bs = ticker.balance_sheet
    y0, y1 = df_bs.loc['Inventory'].iloc[0], df_bs.loc['Inventory'].iloc[1]
    avg_inventory = (y0 + y1)/2

    return ticker.income_stmt.loc['Cost Of Revenue'].iloc[0]/avg_inventory

## Collect Ratios

In [8]:
# Dictionary to collect data to create a DF later
data = {
    'Symbol': [],
    'Name': [],
    'Market Cap': [],
    'EPS fwd': [],
    'PE fwd': [],
    'PEG': [],
    'PB': [],
    'ROE' : [],
    'ROCE' : [],
    'FCFY' : [],
    'D2E' : [],
    'CR' : [],
    'QR' : [],
    'Asset TR': [],
    'DY' : [],
    'Beta': [],
    'Price': [],
    '52w Low': [],
    '52w High': []
    }
industry = ''

for symbol in symbols:
    ticker = yf.Ticker(symbol)
    if not industry:
        industry = ticker.info['industry']
    else:
        industry_current = ticker.info['industry'] 
        if industry_current != industry:
            print(f'Encountred a different industry {industry_current}, previous {industry}. Quitting')
            break        
    populate_with_info(data, ticker.info)
    data['ROCE'].append(roce(ticker))
    data['Asset TR'].append(asset_turnover_ratio(ticker))

## Create DF

In [9]:
# Create a DF using the dictionary
df = pd.DataFrame(data)

# Save any stocks with NaN values
df_exceptions = df[df.isna().any(axis=1)]

# Remove any stocks with NaN values
df=df.dropna()

# Reset index after dropping rows with NaN values
df.reset_index(drop=True, inplace=True)

# Add 52 week price range
df['52w Range'] = ((df['Price'] - df['52w Low'])/(df['52w High'] - df['52w Low']))*100

df_exceptions

,Symbol,Name,Market Cap,EPS fwd,PE fwd,PEG,PB,ROE,ROCE,FCFY,D2E,CR,QR,Asset TR,DY,Beta,Price,52w Low,52w High
1,BIPC,Brookfield Infrastructure Corporation,4.68B,NaN,NaN,NaN,NaN,0.96426,0.437331,-15.31,303.451,0.441,0.441,0.186143,4.31,1.392,35.50,25.48,48.23
4,NJR,New Jersey Resources Corporation,4.1B,2.93,14.313992,2.51,2.055882,0.13904,0.075570,-2.34,NaN,0.660,0.132,0.306744,4.01,0.659,41.94,38.92,55.84


## Score

In [10]:
def score(values, value, cat) -> int:
    '''
    Calculate the score using standard deviation and mean based on the category. A ratio such as PE which prefers a lower
    value, the score is calculated the following way:
    1. Score of 1 is returned if given PE is in between -1 std and mean
    2. Score of 2 is returned if given PE is in between -2 std and -1 std
    3. Score of 3 is returned if PE is outside -2 std
    4. Score of -1 is returned if given PE is in between 1 std and mean
    5. Score of -2 is returned if given PE is in between +1 std and +2 std
    6. Score of -3 is given if given PE is outside +2 std

    A ratio such as ROE which prefers a higher value, the score is calculated the following way:
    1. Score of 1 is returned if given ROE is in between mean and +1 std
    2. Score of 2 is returned if given ROE is in between +1 std and +2 std
    3. Score of 3 is returned if ROE is outside +2 std
    4. Score of -1 is returned if given ROE is in between -1 std and mean
    5. Score of -2 is returned if given ROE is in between -1 std and -2 std
    5. Score of -3 is given if given ROE is outside -2 std

    Parameters
    ----------
    values : List of the values
    value: The value to compare whether it's within mean, 1 std, -1 std, 2 std or -2 std
    cat: Category type, valid value is 1 or 2.
        
    Returns
    -------
    score: the score for given 'value'
    '''
    
    std = statistics.stdev(values)
    mean = statistics.mean(values)

    if cat == 1:
        if (mean + (-1 * std)) < value <= mean:
            return 1
        elif (mean + (-2 * std)) < value <= (mean + (-1 * std)):
            return 2
        elif value <= (mean + (-2 * std)):
            return 3
        elif mean < value <= (mean + (1 * std)):
            return -1
        elif (mean + (1 * std)) < value <= (mean + (2 * std)):
            return -2
        else:
            return -3
    else:
        if mean <= value < (mean + (1 * std)):
            return 1
        elif (mean + (1 * std)) <= value < (mean + (2 * std)):
            return 2
        elif value >= (mean + (2 * std)):
            return 3
        elif (mean + (-1 * std)) <= value < mean:
            return -1
        elif (mean + (-2 * std)) <= value < (mean + (-1 * std)):
            return -2
        else:
            return -3

## Apply scoring

In [11]:
df_score = df.copy()

for col in CAT1_RATIOS:
    for index, value in df[col].items():
        # print(f'{col} - {index} - {value}')
        df_score.loc[index, col] = score(df[col], value, 1)

for col in CAT2_RATIOS:
    for index, value in df[col].items():
        # print(f'{col} - {index} - {value}')
        df_score.loc[index, col] = score(df[col], value, 2)

# Add ranking scores to get the total score
df_score['Score'] = df_score[CAT1_RATIOS+CAT2_RATIOS].sum(axis=1)
# df_score

In [12]:
df

,Symbol,Name,Market Cap,EPS fwd,PE fwd,PEG,PB,ROE,ROCE,FCFY,D2E,CR,QR,Asset TR,DY,Beta,Price,52w Low,52w High,52w Range
0,ATO,Atmos Energy Corporation,17.2B,7.08,16.090395,2.31,1.556220,0.08732,0.053719,2.39,65.528,0.655,0.265,0.191249,2.83,0.667,113.92,101.00,125.28,53.212521
1,CPK,Chesapeake Utilities Corporation,2.29B,5.45,18.871560,2.93,2.111953,0.10473,0.080164,1.61,91.819,0.477,0.234,0.314420,2.29,0.636,102.85,83.80,132.91,38.790470
2,NFE,New Fortress Energy Inc.,6.79B,5.03,6.580516,0.18,4.725871,0.22331,0.045603,-14.30,419.905,0.453,0.315,0.324831,1.21,1.544,33.10,25.06,42.17,46.990064
3,NI,NiSource Inc.,10.6B,1.71,14.947368,1.94,1.743401,0.09476,0.059653,-11.27,170.646,0.413,0.151,0.229915,3.91,0.499,25.56,22.86,28.95,44.334975
4,OGS,"ONE Gas, Inc.",3.4B,3.87,15.821706,2.96,1.282787,0.08935,0.052494,4.88,112.968,0.359,0.119,0.318695,4.31,0.656,61.23,55.50,84.26,19.923505
5,SWX,"Southwest Gas Holdings, Inc.",4.28B,3.07,19.488600,4.52,1.321130,-0.05662,-0.003130,-13.97,157.669,1.899,1.004,0.382100,4.15,0.318,59.83,53.79,68.03,42.415730
6,SR,Spire Inc.,3.13B,4.55,12.929670,2.14,1.169211,0.07584,0.051646,-3.99,162.585,0.613,0.180,0.261437,5.13,0.533,58.83,53.77,75.83,22.937443
7,UGI,UGI Corporation,4.81B,2.92,7.856165,1.38,1.139083,-0.28697,-0.111069,3.58,174.693,0.899,0.522,0.541485,6.54,1.153,22.94,20.19,43.19,11.956522


## Add Styles

In [13]:
def make_pretty(styler):
    # Column formatting
    styler.format({'EPS fwd': '{:.0f}', 'PE fwd': '{:.0f}', 'PEG': '{:.0f}', 'FCFY': '{:.0f}', 'PB' : '{:.0f}', 'ROE' : '{:.0f}',
                   'ROCE': '{:.0f}', 'D2E': '{:.0f}', 'CR': '{:.0f}', 'QR': '{:.0f}', 'Asset TR': '{:.0f}', 'DY': '{:.2f}%',
                   'Beta': '{:.0f}', '52w Low': '${:.2f}', 'Price': '${:.2f}', '52w High': '${:.2f}', '52w Range': '{:.2f}%', 'Score' : '{:.0f}'
                  })

    # Set the bar visualization
    styler.bar(subset = ['52w Range'], align = "mid", color = ["salmon", "cornflowerblue"])

    # Grid
    styler.set_properties(**{'border': '0.1px solid black'})

    # Set background gradients
    for ratio in CAT1_RATIOS:
        styler.background_gradient(subset=[ratio], cmap='RdYlGn', gmap=-df[ratio])
    for ratio in CAT2_RATIOS:
        styler.background_gradient(subset=[ratio], cmap='RdYlGn')
    styler.background_gradient(subset=['Score'], cmap='PiYG')
    
    # Hide index
    styler.hide(axis='index')

    # Left text alignment for some columns
    styler.set_properties(subset=['Symbol', 'Name'], **{'text-align': 'left'})
    styler.set_properties(subset=CAT1_RATIOS + CAT2_RATIOS + ['Market Cap', 'Score'], **{'text-align': 'center'})

    return styler

In [14]:
# Add table caption and styles to DF
df_score.style.pipe(make_pretty).set_caption(f'Stock Screener {industry}').set_table_styles(
    [{'selector': 'th.col_heading', 'props': 'text-align: center'},
     {'selector': 'caption', 'props': [('text-align', 'center'),
                                       ('font-size', '11pt'), ('font-weight', 'bold')]}])

Symbol,Name,Market Cap,EPS fwd,PE fwd,PEG,PB,ROE,ROCE,FCFY,D2E,CR,QR,Asset TR,DY,Beta,Price,52w Low,52w High,52w Range,Score
ATO,Atmos Energy Corporation,17.2B,2,-1,-1,1,1,1,1,1,-1,-1,-2,2.83%,1,$113.92,$101.00,$125.28,53.21%,2
CPK,Chesapeake Utilities Corporation,2.29B,1,-2,-1,-1,1,1,1,1,-1,-1,-1,2.29%,1,$102.85,$83.80,$132.91,38.79%,-1
NFE,New Fortress Energy Inc.,6.79B,1,2,2,-3,2,1,-2,-3,-1,-1,1,1.21%,-2,$33.10,$25.06,$42.17,46.99%,-3
NI,NiSource Inc.,10.6B,-2,-1,1,1,1,1,-1,-1,-1,-1,-1,3.91%,1,$25.56,$22.86,$28.95,44.33%,-3
OGS,"ONE Gas, Inc.",3.4B,-1,-1,-1,1,1,1,2,1,-1,-1,-1,4.31%,1,$61.23,$55.50,$84.26,19.92%,1
SWX,"Southwest Gas Holdings, Inc.",4.28B,-1,-2,-2,1,-1,-1,-2,1,3,3,1,4.15%,2,$59.83,$53.79,$68.03,42.42%,2
SR,Spire Inc.,3.13B,1,1,1,1,1,1,-1,1,-1,-1,-1,5.13%,1,$58.83,$53.77,$75.83,22.94%,4
UGI,UGI Corporation,4.81B,-1,2,1,1,-3,-3,1,-1,1,1,3,6.54%,-2,$22.94,$20.19,$43.19,11.96%,0


## Description
1. __[EPS](https://www.investopedia.com/terms/e/eps.asp)__ (Earnings Per Share): portion of a company’s profit that is assigned to each share of its stock
2. __[PE](https://www.investopedia.com/terms/p/price-earningsratio.asp)__ (Price to Earnings): relationship between the stock price of a company and its per-share earnings. It helps investors determine if a stock is undervalued or overvalued relative to others in the same sector.
3. __[PEG](https://www.investopedia.com/terms/p/pegratio.asp)__ (Projected Earnings Growth): - calculated by dividing a stock’s P/E by its projected 12-month forward revenue growth rate. In general, a PEG lower than 1 is a good sign, and a PEG higher than 2 indicates that a stock may be overpriced
4. __[PB](https://www.investopedia.com/terms/p/price-to-bookratio.asp)__ (Price to Book): A ratio of 1 indicates the company's shares are trading in line with its book value. A P/B higher than 1 suggests the company is trading at a premium to book value, and lower than 1 indicates a stock that may be undervalued relative to the company's assets.
6. __[ROE](https://www.investopedia.com/terms/r/returnonequity.asp)__ (Return on Equity): provides a way for investors to evaluate how effectively a company is using its equity to generate profits. A higher ROE indicates a more efficient use of shareholder equity, which can lead to increased demand for shares and higher stock price, as well as increase in company's profits in the future.
7. __[ROCE](https://www.investopedia.com/terms/r/roce.asp)__ (Return on Capital Employed): measures a company’s profitability in terms of all of its capital
8. __[FCFY](https://www.investopedia.com/articles/fundamental-analysis/09/free-cash-flow-yield.asp)__ (Free Cash Flow Yield): a financial solvency ratio that compares the free cash flow per share a company is expected to earn against its market value per share. A lower ratio indicates a less attractive investment opportunity.
9. __[D2E](https://www.investopedia.com/terms/d/debtequityratio.asp)__ (Debtoi Equity): compares a company's total liabilities with its shareholder equity
10. __[CR](https://www.investopedia.com/terms/c/currentratio.asp)__ (Current Ratio): measures a company's ability to pay off its current liabilities (payable within one year) with its current assets such as cash, accounts receivable, and inventories. The higher the ratio, the better the company's liquidity position.
11. __[QR](https://www.investopedia.com/terms/q/quickratio.asp)__ (Quick Ratio): measures a company's capacity to pay its current liabilities without needing to sell its inventory or obtain additional financing.
12. __[Asset TR](https://en.wikipedia.org/wiki/Asset_turnover)__ (Asset Turnover Ratio): measures the efficiency of a company's assets in generating revenue or sales.
13. __[DY](https://www.investopedia.com/terms/d/dividendyield.asp)__ (Dividend Yield Ratio): ratio looks at the amount paid by a company in dividends every year relative to its share price. It is an estimate of the dividend-only return of a stock investment.
14. __[Beta](https://www.investopedia.com/terms/b/beta.asp)__: is a measure of a stock's volatility in relation to the overall market. A stock that swings more than the market over time has a beta above 1.0. If a stock moves less than the market, the stock's beta is less than 1.0.
15. __52w Range__: a visualization to indicate which stocks are near their 52 week low and which are near their 52 week high. For example, 90% will indicate that the current price is very close to its 52 week high
16. __Score__: sum of ratio scores for each stock